In [ ]:
# First, install OpenFE and other required packages
!pip install openfe
!pip install numpy pandas scikit-learn lightgbm scipy xgboost tqdm pyarrow


In [7]:
from openfe import OpenFE, transform
from sklearn.model_selection import train_test_split
import pandas as pd

btc_data = pd.read_csv('BTC_99.csv')

# Let's assume 'close' is the target variable you want to predict
label = btc_data['close']
data = btc_data.drop(['close'], axis=1)
display(data)



,Unnamed: 0,date,open,high,low,volume
0,0,2023-05-27 00:00:00+00:00,26705.93,26713.40,26686.31,92.77354
1,1,2023-05-27 00:05:00+00:00,26689.14,26694.98,26677.23,139.78788
2,2,2023-05-27 00:10:00+00:00,26692.25,26692.25,26678.60,34.35930
3,3,2023-05-27 00:15:00+00:00,26678.87,26680.56,26673.11,36.52595
4,4,2023-05-27 00:20:00+00:00,26673.12,26675.71,26652.75,61.32347
...,...,...,...,...,...,...
28577,28577,2023-09-03 05:25:00+00:00,25894.77,25897.09,25893.77,18.18597
28578,28578,2023-09-03 05:30:00+00:00,25897.08,25905.00,25897.08,27.96760
28579,28579,2023-09-03 05:35:00+00:00,25903.05,25903.06,25887.63,29.83875
28580,28580,2023-09-03 05:40:00+00:00,25887.63,25887.64,25878.70,25.96047


In [ ]:
# Convert 'date' to datetime and sort by it if needed
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values('date')

# Drop unnecessary columns like 'Unnamed: 0'
data = data.drop(['Unnamed: 0', 'date'], axis=1)

# Split the data into training and testing sets
train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2, random_state=42)

# Initialize OpenFE
ofe = OpenFE()

# Generate new features
features = ofe.fit(data=train_x, label=train_y, n_jobs=14)  # Use all available cores for parallel computing

# Transform the train and test data according to generated features
train_x_transformed, test_x_transformed = transform(train_x, test_x, features, n_jobs=14)

# Now, 'train_x_transformed' and 'test_x_transformed' are ready for training and evaluation.


The number of candidate features is 68
Start stage I selection.


  3%|█▎                                          | 1/34 [00:03<01:41,  3.09s/it]


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Initialize XGBoost model
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror')

# Fit the model on the transformed training data
xgb_model.fit(train_x_transformed, train_y)

# Make predictions on the test data
preds = xgb_model.predict(test_x_transformed)

# Calculate the mean squared error of the model
mse = mean_squared_error(test_y, preds)
print(f"Mean Squared Error: {mse}")

# If you want, you can also calculate other metrics and visualize feature importance


In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

# Cross-validation with 5 folds
cv_scores = cross_val_score(xgb_model, train_x_transformed, train_y, cv=5, scoring='neg_mean_squared_error')
cv_scores = np.sqrt(-cv_scores)  # Convert MSE to RMSE
print(f"Cross-Validation RMSE Scores: {cv_scores}")
print(f"Mean CV RMSE: {np.mean(cv_scores)}")

# Feature Importance
xgb.plot_importance(xgb_model)
plt.show()
